# Identifying Factors for Viral Posts on the Hacker News Website

This project analyzes posts on the *Hacker News* website based on their relevance (upvotes, comments).
Many posts are uploaded either in the *ASK HN (Hacker News)* or *Show HN* categories.
First these two categories are evaluated by their relevance and then the most promising posting time for the favorable category is identified.

The data set can be downloaded [here](https://www.kaggle.com/hacker-news/hacker-news-posts) (external link to kaggle).

#### Summary of Results

The most relevant posts were uploaded in the *Ask HN* category at 15:00 US Eastern Time.

### Cleaning and sorting the data set

First the data set is read in and saved in a `list of lists`:

In [2]:
from csv import reader

opened_file = open("HN.csv")
read_file = reader(opened_file)
hn = list(read_file)

The header is assigned to the variable `headers`, removed from the data set and printed.
The first five rows of the data set are printed as well:



In [3]:
headers = hn[0]
print(headers)
print("\n")

hn = hn[1:]

for i, row in enumerate(hn):
    if i <5:
        print(row)
        print("\n")

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']


['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']


['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']


['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']


['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']




The data can be described as follows:
- id: a unique id
- title: the post's title
- url: if the post contains a url it is put in this row
- num_points: number of upvotes subtracted by the number of downvotes (popularity of the post)
- num_comments: number of comments about the post
- author: the post's author's nickname
- created_at: the time of creation

Data entries with no comments (index=4) are not of interest and will be removed:


In [5]:
hn_cleaner = []

for row in hn:
    num_comments = int(row[4])
    if num_comments != 0:
        hn_cleaner.append(row)
        
#print the number of removed posts:        
delta = len(hn) - len(hn_cleaner)
print(delta)

212718


Furthermore, only the *Ask HN* and *Show HN* posts are of interest in the analysis. So two new `lists` are created by filtering for these two categories:

In [6]:
hn_ask = []
hn_show = []
other_posts = []

for row in hn_cleaner:
    post_title = row[1]
    post_title = post_title.lower() #transform post title to lowercase
    if post_title.startswith('ask hn'):
        hn_ask.append(row)
    elif post_title.startswith('show hn'):
        hn_show.append(row)
    else:
        other_posts.append(row)
        
#print the number of posts in the Ask-category and the first five entries in the new list        
print(len(hn_ask))
for i, row in enumerate(hn_ask):
    if i <5:
        print(row)
        print("\n")
        
#do the same as above for the Show-category        
print(len(hn_show))
for i, row in enumerate(hn_show):
    if i <5:
        print(row)
        print("\n")

6911
['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53']


['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17']


['12577870', 'Ask HN: Why join a fund when you can be an angel?', '', '1', '3', 'anthony_james', '9/25/2016 22:48']


['12577647', 'Ask HN: Someone uses stock trading as passive income?', '', '5', '2', '00taffe', '9/25/2016 21:50']


['12576946', 'Ask HN: How hard would it be to make a cheap, hackable phone?', '', '2', '1', 'hkt', '9/25/2016 19:30']


5059
['12577142', 'Show HN: Jumble  Essays on the go #PaulInYourPocket', 'https://itunes.apple.com/us/app/jumble-find-startup-essay/id1150939197?ls=1&mt=8', '1', '1', 'ryderj', '9/25/2016 20:06']


['12576813', 'Show HN: Learn Japanese Vocab via multiple choice questions', 'http://japanese.vul.io/', '1', '1', 'soulchild37', '9/25/2016 19:06']


['12576090', 'Show HN: Markov chain Twitter bot. Trained on comment

As shown above, the data set is further decreased.

### Analyzing the data set

Now it is analyzed if *Ask HN* oder *Show HN* has more comments on average:

In [4]:
number_ask_comments = 0
number_show_comments = 0

for row in hn_ask:
    num_comments = int(row[4])
    number_ask_comments += num_comments

for row in hn_show:
    num_comments = int(row[4])
    number_show_comments += num_comments
    
avg_number_ask_comments = number_ask_comments / len(hn_ask)   
avg_number_show_comments = number_show_comments / len(hn_show)  
    
print(avg_number_ask_comments)
print(avg_number_show_comments)

percentage = (avg_number_ask_comments - avg_number_show_comments) / avg_number_show_comments
print(percentage)



13.744175951381855
9.810832180272781
0.40091846428869515


It is shown, that the *Ask HN* posts get 40% more comments on average.

In the following the analysis is focused on this category and it will be shown if posts created at a certain time attract more attention.
Consequently a `list` containing the time a post is uploaded accompanied by the number of comments is created:

In [5]:
result_list = []
for row in hn_ask:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

print(result_list[0:5])    

[['9/26/2016 2:53', 7], ['9/26/2016 1:17', 3], ['9/25/2016 22:48', 3], ['9/25/2016 21:50', 2], ['9/25/2016 19:30', 1]]


The date and time a post is created can be parsed with the `datetime` module.
Here only the posts per full hour should be analyzed.
So the *created_at* value is parsed by `datetime` and the hour is extracted:

In [6]:
import datetime as dt

for row in result_list:
    date_and_time = row[0]
    date_time = dt.datetime.strptime(date_and_time, "%m/%d/%Y %H:%M")
    row[0] = date_time.hour
    
print(result_list[0:5])    


[[2, 7], [1, 3], [22, 3], [21, 2], [19, 1]]


Now a `dictionary` is created showing the absolute number of comments per post and full hour. 
Another `dictionary` counts the number of posts per hour:

In [7]:
results = {}
results_counter = {}

for row in result_list:
    hour = row[0]
    num_comments = row[1]
    if hour in results:
        results[hour] += num_comments
        results_counter[hour] += 1
    else:
        results[hour] = num_comments
        results_counter[hour] = 1
        
print(results)
print(results_counter)

{2: 2996, 1: 2089, 22: 3372, 21: 4500, 19: 3954, 17: 5547, 15: 18525, 14: 4972, 13: 7245, 11: 2797, 10: 3013, 9: 1477, 7: 1585, 3: 2154, 16: 4466, 8: 2362, 0: 2277, 23: 2297, 20: 4462, 18: 4877, 12: 4234, 4: 2360, 6: 1587, 5: 1838}
{2: 227, 1: 223, 22: 287, 21: 407, 19: 420, 17: 404, 15: 467, 14: 378, 13: 326, 11: 251, 10: 219, 9: 176, 7: 157, 3: 212, 16: 415, 8: 190, 0: 231, 23: 276, 20: 392, 18: 452, 12: 274, 4: 186, 6: 176, 5: 165}


These two `dictionaries` can be used to calculate the average number of comments per post and hour:

In [8]:
results_list_final = []

for key in results:
    num_comments = results[key]
    num_posts = results_counter[key]
    average_number_comments = num_comments / num_posts
    average_number_comments = round(average_number_comments)
    results[key] = average_number_comments
    results_list_final.append([key, average_number_comments])   

To sort this `list` by decreasing values of average posts the order is swapped:

In [9]:
swapped_results_list_final = []

for row in results_list_final:
    hour = row[0]
    avg_num_comments = row[1]
    swapped_results_list_final.append([avg_num_comments, hour])

sorted_swap = sorted(swapped_results_list_final, reverse = True)   
print(sorted_swap)

[[40, 15], [22, 13], [15, 12], [14, 17], [14, 10], [13, 14], [13, 4], [13, 2], [12, 22], [12, 8], [11, 21], [11, 20], [11, 18], [11, 16], [11, 11], [11, 5], [10, 7], [10, 3], [10, 0], [9, 19], [9, 6], [9, 1], [8, 23], [8, 9]]


The top five hours to post on Hacker News in the *Ask* category based on the number of comments a post attracted are:

In [19]:
for i, row in enumerate(sorted_swap):
    hour = row[1]
    avg_num_comments = row[0]
    if i < 5:
        print("At {:02d}:00 there were {:02d} comments for every post on average".format(hour, avg_num_comments))

At 15:00 there were 40 comments for every post on average
At 13:00 there were 22 comments for every post on average
At 12:00 there were 15 comments for every post on average
At 17:00 there were 14 comments for every post on average
At 10:00 there were 14 comments for every post on average


According to the data set the time zone is Eastern Time in the US (UTC-4).
To convert the data to the summer time zone in Germany (UTC+2: where the author is living at the time of this work) a time delta of six hours has to be introduced.
Consequently the top five hours in german summer time are:

In [20]:
for i, row in enumerate(sorted_swap):
    hour = row[1] + 6
    avg_num_comments = row[0]
    if i < 5:
        print("At {:02d}:00 there were {:02d} comments for every post on average".format(hour, avg_num_comments))

At 21:00 there were 40 comments for every post on average
At 19:00 there were 22 comments for every post on average
At 18:00 there were 15 comments for every post on average
At 23:00 there were 14 comments for every post on average
At 16:00 there were 14 comments for every post on average
